Below are answers to the question, which are part of the assignment described [here.](http://web.stanford.edu/class/cs224n/assignments/a4.pdf)
# 1(g)

    The masks set the input to the softmax for 'pad' tokens to -inf. Then the output of softmax for these tokens equals 0. 
    This cause the tokens to be ignored when calculating attention output. 
    It is necessary because for the attention we care only about actual context of a sentence (represented by values of a hidden state in the Encoder),
    because it brings some information to the model. 'pad' tokens do not brings much relevant information.
    Including 'pad' tokens for short sentences would cause weight assigned to hidden states corresponding to normal words to be smaller. 
# 1(j) 
    * Dot product attention: doesn't introduce additional parameters to the model, requires the 2 vector to be of the same shape 
    * Multiplicative attention: introduce additional parameters to the model, enables to calculate attention between vectors of different shape
    * Additive attention: slower to calculate, more operations, more learnable parameters
# 2(a)
    * Source Sentence: Aqu´ı otro de mis favoritos, “La noche estrellada”.
    Reference Translation: So another one of my favorites, “The Starry Night”.
    NMT Translation: Here’s another favorite of my favorites, “The Starry Night”.
        1. Double favourite.
        2. Two parts of the English sentence with favourite make sense separately so they are highly probable to be returned by a model.
        3. Make a model to notice that a similar word is already in the text and make probability of using it again lower.
        Increase beam size in beam search.

    * Source Sentence: Ustedes saben que lo que yo hago es escribir para los ni˜nos, y,
    de hecho, probablemente soy el autor para ni˜nos, ms ledo en los EEUU.
    Reference Translation: You know, what I do is write for children, and I’m probably America’s
    most widely read children’s author, in fact.
    NMT Translation: You know what I do is write for children, and in fact, I’m probably the
    author for children, more reading in the U.S.
        1. Too direct translation, incorrect gramatically in English.
        2. Model might have put too much weight on specific words.
        3. Increase number of parameters in the model, increase beam size.
    
    * Source Sentence: Un amigo me hizo eso – Richard Bolingbroke.
    Reference Translation: A friend of mine did that – Richard Bolingbroke.
    NMT Translation: A friend of mine did that – Richard <unk>
        1. Unknown token.
        2. Not all words can in included in the vocabulary.
        3. Detect named entitites and copy them directly in the model.
    * Source Sentence: Solo tienes que dar vuelta a la manzana para verlo como una
    epifan´ıa.
    Reference Translation: You’ve just got to go around the block to see it as an epiphany.
    NMT Translation: You just have to go back to the apple to see it as a epiphany.
        1. Sentence does not make sense.
        2. Manzana has 2 meanings: block and apple. The model picked the wrong one.
        3. Use context dependent embeddings of source language. 
    * Source Sentence: Ella salv´o mi vida al permitirme entrar al ba˜no de la sala de
    profesores.
    Reference Translation: She saved my life by letting me go to the bathroom in the teachers’
    lounge.
    NMT Translation: She saved my life by letting me go to the bathroom in the women’s room.
        1. The output sentence does not contain information about teachers.
        2. Model picked the popular term that matched the context. 
    * Source Sentence: Eso es m´as de 100,000 hect´areas.
    Reference Translation: That’s more than 250 thousand acres.
    NMT Translation: That’s over 100,000 acres.
        1. Both translations seems to be incorrect.
        2. Model maybe picked the more common word - acres? 
        


# 2(b)

## Error 1

In [5]:
%%bash
awk NR==61 en_es_data/test.es
awk NR==61 en_es_data/test.en
awk NR==61 outputs/test_outputs.txt

Y as noms, el tmpano muestra un lado diferente de su personalidad.
And just like that,  the iceberg shows you a different side of its personality.
And so the iceberg the iceberg shows a different side of your personality.


1. Error: 'Your' instead of 'its', duplicate of work the iceberg.
2. Reason: Su apart from 'its' is also a formal form of 'your' and 'your personality' seems to be much more reasonable than 'its personality'.
3. Fix: Track somehow the subject of the whole sentence and use this information to decide if 'its' or 'your' is better.

## Error 2

In [6]:
%%bash
awk NR==64 en_es_data/test.es
awk NR==64 en_es_data/test.en
awk NR==64 outputs/test_outputs.txt

Entonces, dos parejas cada una con un beb.
So two couples each conceiving one baby.
So, two couples in a baby.


1. Error: 'In a baby' instead of 'conceiving one baby'.
2. Reason: 'Con' is a preposition but in English the best word in this sentence for it is 'having' or 'conveiving'. 
3. Fix: We need to somehow know that 'in a baby' is rather strange expression - maybe adding beam search run backward (so it would include baby for sure) whould help?   

# 2(c)

In [36]:
from nltk import ngrams
import numpy as np
import re

In [11]:
def n_gram_in_n_grams_cnt(n_gram, n_grams):
    return sum([int(n_gram == target_n_gram) for target_n_gram in n_grams])

In [12]:
def modified_n_gram_precision(candidate_translation, reference_translations, n):
    candidate_n_grams = [n_gram for n_gram in ngrams(candidate_translation.split(), n)]
    references_n_grams = [[n_gram for n_gram in ngrams(r.split(), n)] for r in reference_translations]
    nominator = 0
    for c_n_gram in candidate_n_grams:
        c_n_gram_in_c_cnt = n_gram_in_n_grams_cnt(c_n_gram, candidate_n_grams)
        c_n_gram_in_r_cnts = [n_gram_in_n_grams_cnt(c_n_gram, r_n_grams) for r_n_grams in references_n_grams]
        c_n_gram_in_r_cnt_max = max(c_n_gram_in_r_cnts)
        nominator += min(c_n_gram_in_r_cnt_max, c_n_gram_in_c_cnt)
    return nominator / len(candidate_n_grams)

In [37]:
def words_cnt(sentence):
    return len(re.findall(r'\w+', sentence))

In [49]:
def brevity_penalty(candidate_translation, reference_translations):
    c_len = words_cnt(candidate_translation)
    r_lens = np.asarray([words_cnt(r) for r in reference_translations])
    c_r_diffs = np.abs(r_lens - c_len)
    all_argmin = np.argwhere(c_r_diffs == c_r_diffs.min())
    r_star = r_lens[all_argmin].min()
    if c_len >= r_star:
        return 1
    else:
        return np.exp(1-r_star / c_len)

In [63]:
def BLEU(candidate_translation, reference_translations, lambdas):
    prec_sum = 0
    for i, lamb in enumerate(lambdas):
        n = i+1
        prec_sum += lamb * np.log(modified_n_gram_precision(candidate_translation, reference_translations, n))
    BP = brevity_penalty(candidate_translation, reference_translations)
    return BP * np.exp(prec_sum)

In [64]:
s = 'el amor todo lo puede'
r1 = 'love can always find a way'
r2 = 'love makes anything possible'
c1 = 'the love can always do'
c2 = 'love can make anything possible'

In [65]:
BLEU(c1, [r1, r2], [0.5, 0.5])

0.5477225575051662

In [66]:
BLEU(c2, [r1, r2], [0.5, 0.5])

0.6324555320336759

### Calculated manually 
### c1:

#### 1-grams
* the, love, can, always, do 
* p1: (0 + 1 + 1 + 1 + 0) / 5 = 3/5

#### 2-grams
* the love, love can, can always, always do
* p2: (0 + 1 + 1 + 0) / 4 = 1/2

#### BP
* c: 5
* r*: 4
* c > r* so BP = 1

#### BLEU
1 * exp(0.5 * log (3/5) + 0.5 * log(1/2)) = 0.55

## i)
I agree - second sentence seems to be better translation.

## ii)

In [68]:
BLEU(c1, [r1], [0.5, 0.5])

0.448437301984003

In [70]:
BLEU(c2, [r1], [0.5, 0.5])

0.25890539701513365

Now according to BLEU the first translation is better. I don't agree.

## iii)

Due to data availability, NMT systems are often evaluated with respect to only a
single reference translation. This may be problematic because often the meaning of a sentence can
be expressed in a few different ways, using different words and n-grams. 
With only one single reference translation we miss information that
some n-grams are meaningful if they don't occur in the reference translation.

## iv)

* Advantages of BLEU, compared to human evaluation, as an evaluation metric for Machine Translation:
    * Can be calculated automatically for a new model - so we don't need a human to evaluate the results of a model after it has been trained.
    * We gather the reference translations to be used for BLEU calculation once. Because of this the results are comparable for different models and across time. They don't depend on human evalator, her mood etc.
    * Quick to calculate.
* Disadvantages:
    * Does not take into account whole translation and if it makes sense and sounds reasonable in the output language - because it's based only on the n-grams.
    * Human evaluation should be more accurate, especially where there are not many reference sentences gathered.